**Fetching all USA airports information

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys


from typing import Dict, List
from tqdm import tqdm_notebook
from fake_useragent import UserAgent

In [2]:
url_main = "https://www.flightconnections.com/"
url_codes = "airport-codes"

In [3]:
def get_soup(url):
    res = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    html = res.content
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [4]:
soup = get_soup(url_main + url_codes)

In [11]:
airport_list = soup.findAll("ul", attrs={"class":"airport-list"})
airport_list = [airport_group.findAll("li") for airport_group in airport_list]
airport_list = sum(airport_list, [])

In [12]:
def get_airport_info(airport)->Dict[str,str]:
    link = airport.find("a").get("href")
    code = airport.find("span", attrs={"class":"airport-code"}).text
    name = airport.find("span", attrs={"class":"airport-name"}).text
    city_country = airport.find("span", attrs={"class":"airport-city-country"}).text
    cc = city_country.split(", ")
    city = cc[0]
    country = cc[1]
    return {"link":link, "code":code, "name":name, "city":city, "country":country}

In [13]:
airport_info = []

for airport in tqdm_notebook(airport_list):
    airport = get_airport_info(airport)
    if airport["country"] == "United States":
        airport_info.append(airport)

<ipython-input-13-c2e010333845>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for airport in tqdm_notebook(airport_list):


In [14]:
result = pd.DataFrame(airport_info)
result.to_csv("../data/airport_info.csv")

In [16]:
airport_info = pd.read_csv("../data/airport_info.csv", index_col=0)
airport_info.head()

,link,code,name,city,country
0,/flights-to-allentown-abe,ABE,Lehigh Valley International Airpo,Allentown,United States
1,/flights-to-abilene-abi,ABI,Abilene Regional Airport,Abilene,United States
2,/flights-to-ambler-abl,ABL,Ambler Airport (FAA: AFM),Ambler,United States
3,/flights-to-albuquerque-abq,ABQ,Albuquerque International Sunport,Albuquerque,United States
4,/flights-to-aberdeen-abr,ABR,Aberdeen Regional Airport,Aberdeen,United States


**Visiting every airport link in USA and finding connections in USA